# Šachy - cvičení 01


### pozice na šachovnici

Pozici definujeme řetězcem s následujícími symboly: 
* K - king (král)
* Q - queen (dáma)
* R - rook (věž)
* N - knight (jezdec)
* B - bishop (střelec)
* P - pawn (pěšec)

Velkými písmeny označujeme figury bílé, malými písmeny černé. 

Bude se nám asi hodit definovat "konstantu" obsahující přípustné symboly figur: 

In [7]:
FIGURES = "RNBKQBNRPrnbkqbnrp"

Šachovnici orientujeme v základním postavení nahoře bílé figury, dole černé. Políčka indexujeme od nuly zleva doprava, shora dolů. 

Pro popis postavení figur na šachovnici používáme pozici definovanou řetězcem, který obsahuje jména figur a pomlčky pro prázdná pole. Postupně shora dolů, zleva doprava pro každé políčko jeden znak. 

Např. pro základní postavení a prázdnou šachovnici: 

In [8]:
base_position = "RNBKQBNR" + "P" * 8 + "-" * 32 + "p" * 8 + "rnbkqbnr"
blank_position = "-" * 64
print(base_position)
print(blank_position)

RNBKQBNRPPPPPPPP--------------------------------pppppppprnbkqbnr
----------------------------------------------------------------


Sekvence pomlček můžeme nahradit číslem odpovídajícím počtu pomlček za sebou. V případě jedné pomlčky ponecháme v řetězci (pro přehlednost) pomlčku. 

base_position: "RNBKQBNRPPPPPPPP32pppppppprnbkqbnr"
blank_position: "64"

V programu budeme v komprimované podobě tolerovat vypuštění koncových prázdných polí. Bude-li tedy řetězec generovat méně než 64 polí, zbylá pole považujeme za prázdná. 

Definujme funkci pro převod nekomprimované pozice na komprimovanou. Políčka neobsahující symbol figury budeme považovat za prázdná. 

### unpacked_to_packed_position

In [9]:
FIGURES = "RNBKQBNRPrnbkqbnrp"

def unpacked_to_packed_position(position): 
    packed_position = ""
    number_of_blanks = 0 
    for character in position: 
        if character not in FIGURES: 
            number_of_blanks += 1
        else: 
            # write number of blanks or single blank before figure
            if number_of_blanks: 
                packed_position += "-" if number_of_blanks == 1 else str(number_of_blanks)
                number_of_blanks = 0   # blanks are written
            # write figure
            packed_position += character
    # possible rest in number_of_blanks is ignored
    return packed_position
            
# test: 
position = "RNBKQBNR" + "P" * 8 + "-" * 32 + "p" * 8 + "rnbkqbnr"
print(unpacked_to_packed_position(position))
position = "---KQ---" + "--------" * 6 + "---kq---"
print(unpacked_to_packed_position(position))
position = "---K-Q--" + "--------" * 6 + "---k q--"
print(unpacked_to_packed_position(position))

        

RNBKQBNRPPPPPPPP32pppppppprnbkqbnr
3KQ54kq
3K-Q53k-q


Zkusme napsat také funkci pro opačný převod. V této funkci připustíme speciální pozice "base" a "blank" pro základní postavení a prázdnou šachovnici. 

### packed_to_unpacked_position

In [10]:
FIGURES = "RNBKQBNRPrnbkqbnrp"

def packed_to_unpacked_position(position="base"): 
    if position == "base": 
        position = "RNBKQBNR" + "P" * 8 + "-" * 32 + "p" * 8 + "rnbkqbnr"
    elif position == "blank": 
        position = ""
    number_of_blanks = 0
    unpacked_position = ""
    for character in position: 
        if character.isdigit():
            number_of_blanks = number_of_blanks * 10 + int(character)
            #print(number_of_blanks)
        else:
            unpacked_position += "-" * number_of_blanks 
            number_of_blanks = 0
            unpacked_position += character if character in FIGURES else "-"
    # justify to 64 characters
    unpacked_position = (unpacked_position + "-" * 64)[:64]
    return unpacked_position
        
# test: 
position = "RNBKQBNRPPPPPPPP32pppppppprnbkqbnr"
print(packed_to_unpacked_position(position))
position = "3KQ54kq"
print(packed_to_unpacked_position(position))
position = "3K-Q53k-q"
print(packed_to_unpacked_position(position))
 

RNBKQBNRPPPPPPPP--------------------------------pppppppprnbkqbnr
---KQ------------------------------------------------------kq---
---K-Q-----------------------------------------------------k-q--


## Šachovnice

Když budeme řešit nějakou šachovou úlohu, bude se nám hodit pro vnitřní reprezentaci nějaký lepší model než řetěcec popisující polohu figur na šachovnici. Logicky se nabízí nějaké pole - seznam (seznamů), který budeme indexovat řádkovou a sloupcovou souřadnicí. Uspořádání jsme si stanovili už v úvodu. 

Když budete hledat nějaké informace o seznamech (polích) v Pythonu, pamatujte, že seznam je anglicky list. Seznamy (vícerozměrné) můžeme definovat pomocí (vnořených) cyklů. Obvyklý je zkrácený zápis - list comprehension. 

Prázdnou šachovnici bychom mohli definovat třeba takto: 

In [11]:
from pprint import pprint

ch1 = []     # empty list
ch2 = [[]]   # list of one emty list
ch3 = [[], [], [], [], [], [], [], []]  # list of 8 empty lists
ch3a = [[] for i in range(8)] # as above with list comprehension
ch4 = [["", "", "", "", "", "", "", ""]] # list of one list of 8 empty strins
ch4a = [["" for i in range(8)]] # as above with list comprehension
ch5 = [["" for i in range(8)] for j in range(8)] # list of 8 lists of 8 empty strins
pprint(ch5) # it can be a model of empty chessboard

[['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '']]


Mějme na paměti, že se v seznamech indexuje od nuly! Dle naší dohody o uspořádání by měly být figury v následujícím příkladu umístěny na svou výchozí pozici. (Pokud ne, tak to není proto, že neumím indexovat, ale protože neumím šachy.) 

Aby to lépe vypadalo, jsou prázdná pole reprezentovaná mezerou. 

In [12]:
from pprint import pprint

chessb = [[" " for i in range(8)] for j in range(8)]

chessb[0][3] = "K"
chessb[0][4] = "Q"
chessb[7][3] = "k"
chessb[7][4] = "q"

pprint(chessb)

[[' ', ' ', ' ', 'K', 'Q', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', 'k', 'q', ' ', ' ', ' ']]


Co kdybychom chtěli dostat takovouto krásnou maticovou reprezentaci šachovnice z výše definovaného řetězce reprezentujícího pozici? Tady budeme potřebovat ten nekomprimovaný tvar. Budeme předpokládat korektní řetězec. Pomocí list comprehension bychom to mohli udělat následovně: 

In [13]:
position = "RNBKQBNRPPPPPPPP--------------------------------pppppppprnbkqbnr"
chessb = [list(position[i * 8 : i * 8 + 8]) for i in range(8)]
pprint(chessb)

[['R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 ['-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-'],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'k', 'q', 'b', 'n', 'r']]


Kdybychom chtěli pomlčky v matici nahradit např. mezerami, mohli bychom to udělat vnořeným list comprehension takhle: 

In [14]:
position = "RNBKQBNRPPPPPPPP--------------------------------pppppppprnbkqbnr"
chessb = [[" " if c == "-" else c for c in position[i * 8 : i * 8 + 8]] for i in range(8)]
pprint(chessb)

[['R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'k', 'q', 'b', 'n', 'r']]


Pokud vám ten zápis dloube mozek, tak zapomeňte, že nějaký list comprhension existuje, a pěkně si to udělejte růčo. Napište si vnořené cykly pomocí kterých uděláte vše potřebné. Možná při tom uděláte pár chyb, na kterých se naučíte, jak seznamy fungují, takže to bude i užitečné. Až vás ta ruční práce přestane bavit, tak si vzpomeňte na "list comprehension", ať víte, co googlit. 

Hlavně dejte pozor že např. následující příklad je BLBĚ! Doufám, že mi všichni dokážete říct proč. 

In [15]:
row = [" ", " ", " ", " ", " ", " ", " ", " "]
chessb = []
for i in range(8): 
    chessb.append(row)
    
print("TOHLE JE BLBĚ, I KDYŽ TO TAK MOŽNÁ NEVYPADÁ")
pprint(chessb)
print("THIS IS VERY BAD DESPITE THE PRETTY OUTPUT")

TOHLE JE BLBĚ, I KDYŽ TO TAK MOŽNÁ NEVYPADÁ
[[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]
THIS IS VERY BAD DESPITE THE PRETTY OUTPUT


#### OTÁZKA:

Jaká drobná úprava by stačila, aby to bylo dobře. (I když hnusný by to zůstalo...)

## Třídy a objekty

Asi se nám bude hodit s těmi šachovnicemi pracovat objektově. Vytvoříme šachovnici, budeme k ní mít k dispozici nějaké vlastnosti (třeba rozložení figur) a nějaké metody - funkce, které si předem vymyslíme nebo později domyslíme. 

Jako šikovné vlastnosti se nám jeví řetězcové rozložení figur, a asi i maticová reprezantace, protože se dá očekávat, že s ní budeme pracovat často. Když budeme šachovnici "konstruovat", tak vyjdeme z té řetězcové pozice, kterou můžeme zadat v komprimovaném tvaru. 

V naší třídě můžeme použít i funkce, které jsme si definovali vně této třídy, ale musejí být aktuálně dostupné. Hned na začátku použijeme [packed_to_unpacked_position](#packed_to_unpacked_position). Pokud aktuálně není nadefinována, tak to napravte (spusťte její definici). 

Takže by to mohlo vypadat nějak takhle: 

In [16]:
class Chessboard: 

    def __init__(self, position="base"):
        self.position = position
        self.chessboard = self.get_chessboard()
        
    def get_chessboard(self):
        position = packed_to_unpacked_position(self.position)
        chessboard = [[" " if c == "-" else c 
                       for c in position[i * 8 : i * 8 + 8]] for i in range(8)]
        return chessboard

# test
from pprint import pprint
ch = Chessboard()
print(ch.position)
pprint(ch.chessboard)


base
[['R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R'],
 ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
 ['r', 'n', 'b', 'k', 'q', 'b', 'n', 'r']]


## Vizualizace šachovnice

Pretty print je hezká věc, ale když se budete chtít někomu neznalému pochlubit se svojí šachovnicí, tak s ním mnoho slávy nesklidíte. Lidi mají rádi obrázky. A nejlépe ty na netu. Že jste byli na super dovolené a máte odtamtud pár povedených záběrů? Fajn, pošli mi odkaz! Takže i my se budeme snažit o prezentaci naší práce prostřednictvím webu. Třeba se nakonec i dopracujeme k nějaké plnohodnotné webové aplikaci. Musíme ale postupovat drobnými krůčky, abychom si nedali na hubičku a nebáli se pak ani postavit. 

Co budeme potřebovat? Především obrázky figurek a samotné šachovnice. Budeme používat obrázky vektorové ve formátu SVG, protože současný web má tenhle formát rád. Ty, co jsem zvolil, pocházejí asi odsud: https://commons.wikimedia.org/wiki/Category:SVG_chess_pieces, zvolená orientace má původ, tuším, tady: https://commons.wikimedia.org/wiki/File:AAA_SVG_Chessboard_and_chess_pieces_04.svg. Posléze jsem zjistil, že se asi většinou používá zobrazení otočené o 180 stupňů, ale to je pro naše účely celkem jedno. 

Pokud jste si stáhli cvičný projekt z GitHubu, potřebné obrázky by měly být v adresáři [static/images](static/images/). Jestli je nemáte někde u sebe, tak by měly být u mě na githubu: https://github.com/JerryFox/chess/tree/master/static/images. Jednotlivé obrázky respektive SVG obecně dokáže zobrazit samotný webový prohlížeč, ale budete je muset mít buď u sebe na disku nebo na nějakém webovém serveru. Momentálně jsou třeba na mém cvičném webu http://vysoky.pythonanywhere.com/files/chess/images, ale jejich budoucí existence zde nezaručuji. 

Zkusme vytvořit nový soubor, a zapsat do něj nějaký text. Souboru dáme příponu, která signalizuje, že by se mělo jednat o HTML soubor. 

In [17]:
html = "ahoj lidi"
f = open("pokus.htm", "w") 
f.write(html)
f.close()

Pokud se zápis podařil, měl by existovat soubor [pokus.htm](pokus.htm), a měl by jít otevřít v prohlížeči. V prohlížeči zkuste zmáčknout ctrl + u, abyste zobrazili zdrojový kód stránky. Uvidíte, že ve zdrojovém kódu je přesně to, co jsme zapsali do souboru. Nic víc, nic méně.  

Kdybychom chtěli vygenerovat validní html stránku, s výhodou použijeme víceřádkový řetězec a do něj vložíme potřebný HTML kód: 

In [18]:
html = """<!DOCTYPE html>
<html>
	<head>
		<title>vygenerovaná stránka</title>
		<meta charset=utf-8">
	</head>
	<body>
        <h1>Skupina Brutus</h1>
        <h2>Dlažební kostka</h2>
        <p>Šel pes do lesa a potkal dlažební kostku.<br>
        Dlažební kostko, kam jdeš? Já jdu do lesa.<p>
	</body>
</html>

"""
f = open("pokus.htm", "w") 
f.write(html)
f.close()

A zkuste soubor [pokus.htm](pokus.htm) znovu otevřít. 

Podívejte se na soubor [chessboard_ntb.htm](chessboard_ntb.htm). Jestli se zobrazuje dobře i s obrázky, tak bezva. Pokud ne, nejsou obrázky tam, kde je kód očekává. Na šachovnici je jenom pár figur, aby nebyl kód (ctrl + u) moc dlouhý. SVG je vlastně XML, tedy text, který může být v samostatném souboru nebo jako v tomto případě může být vložen přímo do HTML. 

### SVG - Scalable Vector Grafic

V tomto souboru je jeden element SVG a v něm jsou vloženy elementy IMAGE reprezentující obrázek šachovnice a obrázky jednotlivých figur. Momentálně nemá smysl řešit detailně všechny atributy a jejich hodnoty, ale zaměříme se na ty pro nás důležité, což jsou x a y souřadnice (v SVG zleva doprava, shora dolů), width, height a href, tedy jméno souboru s obrázkem a cesta k němu. Figura vlevo nahoře má nějaký x0 a y0 posun oproti šachovnici. Rozměry políčka jsou 100 x 100, takže souřadnice ostatních figur jsou x0 + 100 x sloupec a y0 + 100 x řádek. Připomínám, že řádky a sloupce číslujeme od nuly. 

Jméno souboru má strukturu Chess_<figura><barva>t45.svg. 

* figura - symbol figury (k, q, r, n, b, p)
* barva - l - light (bílá), d - dark (černá)

Cesta k obrázkům může být relativní, absolutní na stejném serveru nebo kompletní URL. 

My používáme jako symboly figur malá písmena pro černé a velká pro bílé. Jména figur opovídají znakům použitým ve jménech souborů. Zkuste napsat funkci, která dostane na vstupu symbol, který používáme v naší maticové reprezentaci šachovnice a bude vracet jméno souboru: 

In [19]:
FIGURES = "RNBKQBNRPrnbkqbnrp"

def get_img_name(shortcut):
    """file name of figure shortcut's image
    shortcuts:
    RNBKQBNRP - dark figures (d)
    rnbkqbnrp - light figures (l)
    image name format example:
    Chess_klt45.svg - white king
    """
    name = "Chess_{}{}t45.svg".format(shortcut.lower(),
                                      "l" if shortcut.isupper() else "d")
    return name

# test:
print(get_img_name("R"))
print(get_img_name("r"))


Chess_rlt45.svg
Chess_rdt45.svg


Dále je vhodné napsat funkci, která bude vracet kód pro celý element odpovídající figuře na šachovnici:

In [20]:
CHESS_IMG_FOLDER = "static/images/"

def image_element_code(row, column, shortcut):
    image_template = """<image x="{}" y="{}" preserveAspectRatio="xMinYMin" xmlns:xlink="http://www.w3.org/1999/xlink"
xlink:href="{}" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0); "
width="80" height="80"></image>
"""
    if shortcut in FIGURES: 
        return image_template.format(15 + column * 100, 13 + row * 100,
                       CHESS_IMG_FOLDER + get_img_name(shortcut))
    else: 
        return ""

# test:
print(image_element_code(3,0,"K"))
print(image_element_code(3,0," "))

<image x="15" y="313" preserveAspectRatio="xMinYMin" xmlns:xlink="http://www.w3.org/1999/xlink"
xlink:href="static/images/Chess_klt45.svg" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0); "
width="80" height="80"></image>




A teď ještě potřebujeme vygenerovat kód pro celý SVG element. Ten už ale musíme generovat na základě celé šachovnice. Parametrem generující funkce tedy bude maticová reprezentace šachovnice: 

In [21]:
def svg_element_code(chessboard):
    svg_template = """<svg class="chessboard" height="810" version="1.1" width="810" \
xmlns="http://www.w3.org/2000/svg" style="overflow: hidden; position: relative;">
<image x="0" y="0" width="810" height="810" preserveAspectRatio="xMinYMin" \
    xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="{img_folder}Chess_Board_01.svg"
    style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);"></image>
{images}</svg>
"""
    images = ""
    for row in range(len(chessboard)):
        for column in range(len(chessboard[0])):
            if chessboard[row][column]:
                images += image_element_code(row, column, chessboard[row][column])
    return svg_template.format(img_folder=CHESS_IMG_FOLDER, images=images)

# test: 
chessboard = [[' ', ' ', ' ', 'K', 'Q', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]
print(svg_element_code(chessboard))


<svg class="chessboard" height="810" version="1.1" width="810" xmlns="http://www.w3.org/2000/svg" style="overflow: hidden; position: relative;">
<image x="0" y="0" width="810" height="810" preserveAspectRatio="xMinYMin"     xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="static/images/Chess_Board_01.svg"
    style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);"></image>
<image x="315" y="13" preserveAspectRatio="xMinYMin" xmlns:xlink="http://www.w3.org/1999/xlink"
xlink:href="static/images/Chess_klt45.svg" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0); "
width="80" height="80"></image>
<image x="415" y="13" preserveAspectRatio="xMinYMin" xmlns:xlink="http://www.w3.org/1999/xlink"
xlink:href="static/images/Chess_qlt45.svg" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0); "
width="80" height="80"></image>
</svg>



A teď už je jenom potřeba svg element obalit HTML kódem, abychom dostali regulérní HTML element (soubor): 

In [22]:
def html_source_code(insert_html):
    html_template = """<!DOCTYPE html>
<html>
	<head>
		<title>chessboard</title>
		<meta charset=utf-8">
	</head>
	<body>
{insert_html}
	</body>
</html>
"""
    return html_template.format(insert_html=insert_html)

# test: 
print(html_source_code("<h1>Šel chlapeček na kopeček.</h1>"))



<!DOCTYPE html>
<html>
	<head>
		<title>chessboard</title>
		<meta charset=utf-8">
	</head>
	<body>
<h1>Šel chlapeček na kopeček.</h1>
	</body>
</html>



A teď to můžeme vyzkoušet všechno dohromady. Vytvoříme objek třídy Chessboard() a vizualizujeme ho pomocí HTML kódu, který opět uložíme do souboru [pokus.htm](pokus.htm). Nezapomeňte, že budeme potřebovat nadefinovanou třídu a spoustu funkcí uvedených výše. Pokud používáte interaktivní notebook zvolit v menu Cell volbu Run All. 

In [23]:
CHESS_IMG_FOLDER = "static/images/"
ch = Chessboard("3KQ2R")
f = open("pokus.htm", "w")
html = html_source_code(svg_element_code(ch.chessboard))
f.write(html)
f.close()

Tak mně to funguje. Doufám, že i vám. Pokračovat budeme tím, že se pokusíme vytvořit serverovou aplikaci, která nám bude ty šachovnice generovat na základě URL adresy, kterou vložíme do adresního řádku prohlížeče, a v níž bude požadované rozložení figur uvedeno formou komprimované pozice, kterou jsme si vymysleli někde [tady (unpacked_to_packed_position)](#unpacked_to_packed_position).

A ještě poslední věc: Všechno z tohoto cvičení dám do souboru [chess_exercise01.py](chess_exercise01.py), takže to bude možné naimportovat a použít: 

In [24]:
import chess_exercise01 as chess
ch = chess.Chessboard("3KQ2RPPPP")
f = open("pokus.htm", "w")
html = ch.get_html()
f.write(html)
f.close()

Tohle mi taky funguje, takže tuhle část tím skutečně uzavírám, a podle času a sil se pustím do té další. Měla by to být [chess_exercise02.ipynb](chess_exercise02.ipynb).